In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime as dt, date, timedelta
from polygon import RESTClient
from secret import API_KEY

In [2]:
# Getting API key and client
api_key = API_KEY
client = RESTClient(api_key)

In [3]:
# Importing buy/sell signals
data = pd.read_csv('signals.csv')

In [4]:
# Creating columns for prices purchased and sold at 
data['call_prices'] = np.zeros(len(data))
data['put_prices'] = np.zeros(len(data))

In [5]:
# Setting ticker
underlying_ticker = 'SPY'

In [6]:
def get_option_ticker(timestamp, price, type):
    # Fixing timestamps
    date = pd.to_datetime(timestamp).date()
    date_more = date + timedelta(weeks=1)

    try:
        # Initialize a list to collect all contract data
        options_list = []

        # Fetch options data
        options_data = client.list_options_contracts(
            underlying_ticker=underlying_ticker,
            expiration_date_lte=date_more,
            expiration_date_gte=date,
            strike_price_gte=(price),
            strike_price_lte=(price+10),
            contract_type=type,
            limit=1000,
            expired=True
        )

        # Loop through the options data and append each contract's data to options_list
        for option in options_data:
            options_list.append(option.__dict__)

        # Convert the list of options data to a DataFrame
        df = pd.DataFrame(options_list)

        # Finding closest ticker
        option_ticker = df['ticker'].iloc[0]

        return option_ticker
    except Exception as e:
        print(f'Error: {e}')

    return None

def get_option_data(timestamp, ticker):
    date = pd.to_datetime(timestamp).date()

    try:
        aggs = client.get_aggs(
                ticker=ticker,
                multiplier=1,
                timespan="minute",
                from_=date,
                to=date,
                limit=5000
            )
        
        df = pd.DataFrame([agg.__dict__ for agg in aggs])

        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df['timestamp'] = df['timestamp'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')
        
        # Assuming df is your DataFrame with a 'timestamp' column
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)

        # Resample to ensure no missing minute intervals
        df = df.resample('1T').ffill()  # Forward-fill missing rows

        df.reset_index(inplace=True, drop=False)
        df['time'] = df['timestamp'].dt.time
        df['time'] = df['time'].apply(lambda t: t.strftime("%H:%M:%S"))
        
        return df
    except Exception as e:
        print(f"Error retreieving data: {e}")

    return None


In [8]:
buy_call_price = None
buy_put_price = None
sell_call_price = None
sell_put_price = None
option_price_data = None

for index, row in data.iterrows():
    if row['call_buy'] and row['call_prices'] == 0:
        timestamp = row['timestamp']
        ticker = get_option_ticker(timestamp, row['close'], 'call')
        option_price_data = get_option_data(timestamp, ticker)

        for i in range(0,1):
            try:
                row_time = (pd.to_datetime(row['time']) + timedelta(minutes=i)).strftime("%H:%M:%S")


                buy_call_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
                data.at[index, 'call_prices'] = buy_call_price
                
                print(index)
                break

            except Exception as e:
                print(f'No call buy price at: {timestamp}\nError: {e}')
                data.at[index, 'call_prices'] = None
                continue

    elif row['call_sell'] and row['call_prices'] == 0:
        for i in range(0,1):
            try:
                row_time = (pd.to_datetime(row['time']) + timedelta(minutes=i)).strftime("%H:%M:%S")


                sell_call_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
                data.at[index, 'call_prices'] = sell_call_price
                
                print(index)
                break

            except Exception as e:
                print(f'No call sell price at: {timestamp}\nError: {e}')
                data.at[index, 'call_prices'] = None
                continue

    elif row['put_buy'] and row['put_prices'] == 0:
        timestamp = row['timestamp']
        ticker = get_option_ticker(timestamp, row['close'], 'put')
        option_price_data = get_option_data(timestamp, ticker)

        for i in range(0,1):
            try:
                row_time = (pd.to_datetime(row['time']) + timedelta(minutes=i)).strftime("%H:%M:%S")


                buy_put_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
                data.at[index, 'put_prices'] = buy_put_price
                
                print(index)
                break
                
            except Exception as e:
                print(f'No put buy price at: {timestamp}\nError: {e}')
                data.at[index, 'put_prices'] = None
                continue

    elif row['put_sell'] and row['put_prices'] == 0:
        for i in range(0,1):
            try:
                row_time = (pd.to_datetime(row['time']) + timedelta(minutes=i)).strftime("%H:%M:%S")


                sell_put_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
                data.at[index, 'put_prices'] = sell_put_price
                
                print(index)
                break

            except Exception as e:
                print(f'No put sell price at: {timestamp}\nError: {e}')
                data.at[index, 'put_prices'] = None
                continue

4
5
6
7
8
9
10
11


KeyboardInterrupt: 

In [ ]:
data.to_csv('final_data.csv')

In [ ]:
null_list = (data[(data.isna()['call_prices'] | data.isna()['put_prices']) == True].index).to_list()

for row_index in null_list:
    if row_index % 2 == 0:
        data = (data.drop([row_index, (row_index+1)])).copy()
        try:
            null_list.remove(row_index+1)
        except:
            continue
    else:
        data = (data.drop([(row_index -1), row_index])).copy()

In [ ]:
data.to_csv('backtest_data.csv')